In [41]:
import string
import urllib.request
from pathlib import Path

import nltk
import polars as pl
import pymorphy3
from navec import Navec
from razdel import tokenize
from slovnet import Morph


# 1. Выбрать dataset с предложениями на русском языке

In [2]:
df = pl.read_csv(
    "https://github.com/0xFEE1DEAD/tatoeba_rus_to_eng/raw/refs/heads/main/tatoeba_rus_to_eng.tsv",
    separator="\t",
    has_header=False,
    new_columns=["ru_id", "ru", "eng_id", "eng"],
    encoding="utf8",
    quote_char=None,
    ignore_errors=True,
    truncate_ragged_lines=True,
    n_rows=15000,
)

# С помощью NLTK разбейте предложения на токены

In [ ]:
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/pavel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/pavel/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pavel/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
df_with_tokens = df.with_columns(
    pl.col("ru")
    .map_elements(lambda sentence: nltk.word_tokenize(sentence, language="russian"), return_dtype=pl.List(pl.String))
    .alias("tokens"),
)

In [7]:
df_with_tokens

ru_id,ru,eng_id,eng,tokens
i64,str,i64,str,list[str]
243,"""Один раз в жизни я делаю хорош…",3257,"""For once in my life I'm doing …","[""Один"", ""раз"", … "".""]"
5409,"""Давайте что-нибудь попробуем!""",1276,"""Let's try something.""","[""Давайте"", ""что-нибудь"", … ""!""]"
5410,"""Мне пора идти спать.""",1277,"""I have to go to sleep.""","[""Мне"", ""пора"", … "".""]"
5411,"""Что ты делаешь?""",16492,"""What are you doing?""","[""Что"", ""ты"", … ""?""]"
5411,"""Что ты делаешь?""",511884,"""What do you make?""","[""Что"", ""ты"", … ""?""]"
…,…,…,…,…
615493,"""Где туалет?""",2136,"""Where is the bathroom?""","[""Где"", ""туалет"", ""?""]"
615493,"""Где туалет?""",634087,"""Where's the bathroom?""","[""Где"", ""туалет"", ""?""]"
615493,"""Где туалет?""",687556,"""Where are the toilets?""","[""Где"", ""туалет"", ""?""]"


# 3. Приведите токены к строчным (lowercase) буквам

In [11]:
def lower_in_list(tokens: list[str]) -> list[str]:
    return list(map(lambda sentence: str.lower(sentence), tokens))


df_with_tokens = df_with_tokens.with_columns(
    pl.col("tokens").map_elements(lambda tokens: lower_in_list(tokens), return_dtype=pl.List(pl.String)),
)
df_with_tokens

ru_id,ru,eng_id,eng,tokens
i64,str,i64,str,list[str]
243,"""Один раз в жизни я делаю хорош…",3257,"""For once in my life I'm doing …","[""один"", ""раз"", … "".""]"
5409,"""Давайте что-нибудь попробуем!""",1276,"""Let's try something.""","[""давайте"", ""что-нибудь"", … ""!""]"
5410,"""Мне пора идти спать.""",1277,"""I have to go to sleep.""","[""мне"", ""пора"", … "".""]"
5411,"""Что ты делаешь?""",16492,"""What are you doing?""","[""что"", ""ты"", … ""?""]"
5411,"""Что ты делаешь?""",511884,"""What do you make?""","[""что"", ""ты"", … ""?""]"
…,…,…,…,…
615493,"""Где туалет?""",2136,"""Where is the bathroom?""","[""где"", ""туалет"", ""?""]"
615493,"""Где туалет?""",634087,"""Where's the bathroom?""","[""где"", ""туалет"", ""?""]"
615493,"""Где туалет?""",687556,"""Where are the toilets?""","[""где"", ""туалет"", ""?""]"


# 4. С помощью pymorphy3 выделите в токенах леммы (нормальные формы слов)

In [18]:
morph = pymorphy3.MorphAnalyzer(lang="ru")


def get_lemmas(tokens: list[str]) -> list[str]:
    return list(map(lambda token: morph.parse(token)[0].normal_form, tokens))


df_with_tokens = df_with_tokens.with_columns(
    pl.col("tokens").map_elements(lambda tokens: get_lemmas(tokens), return_dtype=pl.List(pl.String)),
)
df_with_tokens

ru_id,ru,eng_id,eng,tokens
i64,str,i64,str,list[str]
243,"""Один раз в жизни я делаю хорош…",3257,"""For once in my life I'm doing …","[""один"", ""раз"", … "".""]"
5409,"""Давайте что-нибудь попробуем!""",1276,"""Let's try something.""","[""давать"", ""что-нибудь"", … ""!""]"
5410,"""Мне пора идти спать.""",1277,"""I have to go to sleep.""","[""я"", ""пора"", … "".""]"
5411,"""Что ты делаешь?""",16492,"""What are you doing?""","[""что"", ""ты"", … ""?""]"
5411,"""Что ты делаешь?""",511884,"""What do you make?""","[""что"", ""ты"", … ""?""]"
…,…,…,…,…
615493,"""Где туалет?""",2136,"""Where is the bathroom?""","[""где"", ""туалет"", ""?""]"
615493,"""Где туалет?""",634087,"""Where's the bathroom?""","[""где"", ""туалет"", ""?""]"
615493,"""Где туалет?""",687556,"""Where are the toilets?""","[""где"", ""туалет"", ""?""]"


# 5.Очистите токены от стоп-слов и знаков препинания

In [28]:
clean_set = set(nltk.corpus.stopwords.words("russian")) | set(string.punctuation)


def clean_tokens(tokens: list[str]) -> list[str]:
    return list(filter(lambda token: token not in clean_set, tokens))


df_with_tokens = df_with_tokens.with_columns(
    pl.col("tokens").map_elements(lambda tokens: clean_tokens(tokens), return_dtype=pl.List(pl.String)),
)
df_with_tokens

ru_id,ru,eng_id,eng,tokens
i64,str,i64,str,list[str]
243,"""Один раз в жизни я делаю хорош…",3257,"""For once in my life I'm doing …","[""жизнь"", ""делать"", … ""бесполезно""]"
5409,"""Давайте что-нибудь попробуем!""",1276,"""Let's try something.""","[""давать"", ""что-нибудь"", ""попробовать""]"
5410,"""Мне пора идти спать.""",1277,"""I have to go to sleep.""","[""пора"", ""идти"", ""спать""]"
5411,"""Что ты делаешь?""",16492,"""What are you doing?""","[""делать""]"
5411,"""Что ты делаешь?""",511884,"""What do you make?""","[""делать""]"
…,…,…,…,…
615493,"""Где туалет?""",2136,"""Where is the bathroom?""","[""туалет""]"
615493,"""Где туалет?""",634087,"""Where's the bathroom?""","[""туалет""]"
615493,"""Где туалет?""",687556,"""Where are the toilets?""","[""туалет""]"


# 6.Выберите одно предложение и с помощью pymorphy и Natasha проведите морфологический анализ входящих в него слов (часть речи, род, число, падеж, время и т.п.)

In [31]:
sentence = df["ru"][0]
sentence

'Один раз в жизни я делаю хорошее дело... И оно бесполезно.'

In [ ]:
morph = pymorphy3.MorphAnalyzer(lang="ru")
punct = set(string.punctuation)

print(f"{'Слово':<12} {'Часть речи':<12} {'Род':<8} {'Число':<8} {'Падеж':<10} {'Время':<8} {'Доп.':<20}")
print("-" * 75)

for word in nltk.word_tokenize(sentence, language="russian"):
    if word in punct:
        continue

    parse = morph.parse(word)[0]
    tag = parse.tag

    pos = str(tag.POS).lower() if tag.POS else ""
    gender = str(tag.gender).lower() if tag.gender else ""
    number = str(tag.number).lower() if tag.number else ""
    case = str(tag.case).lower() if tag.case else ""
    tense = str(tag.tense).lower() if tag.tense else ""

    extra = []
    if tag.animacy:
        extra.append(f"одуш={tag.animacy}")
    if tag.aspect:
        extra.append(f"вид={tag.aspect}")
    if tag.voice:
        extra.append(f"залог={tag.voice}")

    print(f"{word:<12} {pos:<12} {gender:<8} {number:<8} {case:<10} {tense:<8} {' '.join(extra):<20}")


Слово        Часть речи   Род      Число    Падеж      Время    Доп.                
---------------------------------------------------------------------------
Один         adjf         masc     sing     nomn                                    
раз          noun         masc     plur     gent                одуш=inan           
в            prep                                                                   
жизни        noun         femn     sing     gent                одуш=inan           
я            npro                  sing     nomn                                    
делаю        verb                  sing                pres     вид=impf            
хорошее      adjf         neut     sing     accs                                    
дело         noun         neut     sing     nomn                одуш=inan           
...                                                                                 
И            conj                                                         

In [42]:
def ensure_model(url: str, local_path: str) -> None:
    local_path = Path(local_path)
    if not local_path.exists():
        print(f"Скачивание {url} → {local_path}")
        local_path.parent.mkdir(parents=True, exist_ok=True)
        urllib.request.urlretrieve(url, local_path)

In [48]:
ensure_model(
    "https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar",
    "natasha/navec_news_v1_1B_250K_300d_100q.tar",
)

Скачивание https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar → natasha/navec_news_v1_1B_250K_300d_100q.tar


In [ ]:
ensure_model(
    "https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_morph_news_v1.tar",
    "natasha/slovnet_morph_news_v1.tar",
)

Скачивание https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_morph_news_v1.tar → slovnet_morph_news_v1.tar


In [54]:
navec = Navec.load("natasha/navec_news_v1_1B_250K_300d_100q.tar")
morph = Morph.load("natasha/slovnet_morph_news_v1.tar")
morph.navec(navec)

tokens = [_.text for _ in tokenize(sentence)]
markups = morph(tokens)

for markup in markups:
    for word in markup:
        print(f"{word.text:<12} {word.tag}")

Один         NUM|Animacy=Inan|Case=Acc|Gender=Masc
раз          NOUN|Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing
в            ADP
жизни        NOUN|Animacy=Inan|Case=Loc|Gender=Fem|Number=Sing
я            PRON|Case=Nom|Number=Sing|Person=1
делаю        VERB|Aspect=Imp|Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin|Voice=Act
хорошее      ADJ|Animacy=Inan|Case=Acc|Degree=Pos|Gender=Neut|Number=Sing
дело         NOUN|Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing
...          PUNCT
И            CCONJ
оно          PRON|Case=Nom|Gender=Neut|Number=Sing|Person=3
бесполезно   ADJ|Degree=Pos|Gender=Neut|Number=Sing|Variant=Short
.            PUNCT
